In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [26]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count_filtered5.csv')

In [4]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_embeddings_full.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [0.012072664, 0.17292306, 0.0061238254, 0.0707...         2
1  [-0.17554894, 0.24209566, 0.4195969, -0.185033...         8
2  [-0.096951924, 0.0034472912, 0.005701333, 0.01...      1524
3  [-0.21775067, 0.244962, 0.24090661, 0.1647732,...      1785
4  [-0.069424234, -0.016805744, 0.21406727, -0.27...      1787


In [7]:
df2 = df_embedding.copy()

In [16]:
import numpy as np
# 임베딩 차원
embedding_dim = 768

# 각 행에 대해 랜덤한 768차원의 리스트 생성 후 DataFrame에 저장
for index, row in df_embedding.iterrows():
    random_embedding = list(2*np.random.rand(embedding_dim)-1)
    df_embedding.at[index,'embedding'] = random_embedding

print(df_embedding)

                                               embedding  track_id
0      [0.9835080643590668, -0.33255553657026415, 0.9...         2
1      [-0.17782470498074243, 0.751982682719927, -0.0...         8
2      [-0.20044520356856044, -0.9508445998080441, 0....      1524
3      [0.045473939251618356, 0.4205875489161761, 0.0...      1785
4      [-0.7551134731854021, 0.8125364091647753, -0.3...      1787
...                                                  ...       ...
31105  [-0.17892181738939295, 0.9273685773866227, -0....  13430827
31106  [0.40350073640095596, 0.44912694561989785, -0....  13430837
31107  [-0.2954503155221524, -0.6623476741430891, 0.3...  13430948
31108  [-0.5975362836597187, 0.2915156522015987, -0.1...  13430963
31109  [-0.7369895345134674, 0.5311878110411827, 0.23...  13430969

[31110 rows x 2 columns]


In [27]:
print(df.shape)
df = pd.merge(df, df_embedding, on='track_id', how='inner')
df.shape

(4645010, 4)


(1294301, 5)

In [28]:
df.head()

,Unnamed: 0,user_id,track_id,listen_count_bin,embedding
0,0,21072247,2,5,"[0.26222911569426643, 0.5532373412623375, 0.87..."
1,1,32226961,2,5,"[0.26222911569426643, 0.5532373412623375, 0.87..."
2,2,2115525,2,5,"[0.26222911569426643, 0.5532373412623375, 0.87..."
3,3,4812967,2,3,"[0.26222911569426643, 0.5532373412623375, 0.87..."
4,4,21211675,2,1,"[0.26222911569426643, 0.5532373412623375, 0.87..."


In [29]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])
df_embedding['track_id'] = lyrics_encoder.fit_transform(df_embedding['track_id'])

In [30]:
lyrics_dict = dict(zip(df_embedding['track_id'], df_embedding['embedding']))

In [31]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    lyrics_embeddings = torch.tensor(np.vstack(dataset['embedding'].values), dtype=torch.float)
    return users, items, ratings, lyrics_embeddings

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_users, train_items, train_ratings, train_lyrics_embeddings = df_to_tensor(train_df)
test_users, test_items, test_ratings,test_lyrics_embeddings = df_to_tensor(test_df)

train_data = TensorDataset(train_users, train_items, train_ratings,train_lyrics_embeddings)
test_data = TensorDataset(test_users, test_items, test_ratings,test_lyrics_embeddings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

23354
1856


In [32]:
# NCF 모델 정의
class NCF_embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF_embedding, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.lyrics_embedding = nn.Linear(768, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 3, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item, lyrics_embedding):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        lyrics_embedding = self.lyrics_embedding(lyrics_embedding.reshape(lyrics_embedding.shape[0],768))
        x = torch.cat((user_embedding, item_embedding,lyrics_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_embedding(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF_embedding(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating,lyrics_embedding in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating,lyrics_embedding = user.to(device), item.to(device), rating.to(device),lyrics_embedding.to(device)  # GPU로 이동
          output = model(user, item,lyrics_embedding.float())
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

      # 각 에폭이 끝날 때마다 테스트 데이터에 대한 예측 수행
      model.eval()
      all_predictions = []
      with torch.no_grad():
          for user, item, _,lyrics_embedding in tqdm(test_loader, desc=f'Testing Epoch {epoch+1}'):
              user, item,lyrics_embedding = user.to(device), item.to(device),lyrics_embedding.to(device)  # GPU로 이동
              output = model(user, item,lyrics_embedding)
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
      print(f'Epoch {epoch+1}/{num_epochs}, RMSE on test set: {rmse}')


Using device: cuda


In [33]:
train_embedding(64,15)

Epoch 1/15: 100%|██████████| 4045/4045 [00:30<00:00, 130.71it/s]


Epoch 1/15, Avg. Loss: 1.7147


Testing Epoch 1: 100%|██████████| 1012/1012 [00:05<00:00, 194.76it/s]


Epoch 1/15, RMSE on test set: 1.2454082397821742


Epoch 2/15: 100%|██████████| 4045/4045 [00:28<00:00, 141.33it/s]


...

Epoch 14/15, RMSE on test set: 1.2068944843095573


Epoch 15/15: 100%|██████████| 4045/4045 [00:28<00:00, 142.60it/s]


Epoch 15/15, Avg. Loss: 0.9811


Testing Epoch 15: 100%|██████████| 1012/1012 [00:05<00:00, 194.40it/s]

Epoch 15/15, RMSE on test set: 1.2106078002890461


In [34]:
train_embedding(256,15)

Epoch 1/15: 100%|██████████| 4045/4045 [00:34<00:00, 117.20it/s]


Epoch 1/15, Avg. Loss: 1.6477


Testing Epoch 1: 100%|██████████| 1012/1012 [00:04<00:00, 220.80it/s]


Epoch 1/15, RMSE on test set: 1.2255125869124537


Epoch 2/15: 100%|██████████| 4045/4045 [00:35<00:00, 113.73it/s]


...

Epoch 14/15, RMSE on test set: 1.2077944562412237


Epoch 15/15: 100%|██████████| 4045/4045 [00:35<00:00, 114.52it/s]


Epoch 15/15, Avg. Loss: 0.5526


Testing Epoch 15: 100%|██████████| 1012/1012 [00:04<00:00, 217.07it/s]

Epoch 15/15, RMSE on test set: 1.2147670948562048


In [35]:
train_embedding(512,15)

Epoch 1/15: 100%|██████████| 4045/4045 [00:43<00:00, 92.10it/s]


Epoch 1/15, Avg. Loss: 1.6105


Testing Epoch 1: 100%|██████████| 1012/1012 [00:05<00:00, 194.54it/s]


Epoch 1/15, RMSE on test set: 1.2222127753059537


Epoch 2/15: 100%|██████████| 4045/4045 [00:44<00:00, 91.18it/s]


...

Epoch 14/15, RMSE on test set: 1.203021504619568


Epoch 15/15: 100%|██████████| 4045/4045 [00:44<00:00, 90.22it/s]


Epoch 15/15, Avg. Loss: 0.4033


Testing Epoch 15: 100%|██████████| 1012/1012 [00:05<00:00, 191.98it/s]

Epoch 15/15, RMSE on test set: 1.2282612626862939


In [36]:
train_embedding(768,15)

Epoch 1/15: 100%|██████████| 4045/4045 [00:55<00:00, 72.62it/s]


Epoch 1/15, Avg. Loss: 1.5993


Testing Epoch 1: 100%|██████████| 1012/1012 [00:05<00:00, 189.75it/s]


Epoch 1/15, RMSE on test set: 1.2209873609982442


Epoch 2/15: 100%|██████████| 4045/4045 [00:55<00:00, 72.83it/s]


...

Epoch 14/15, RMSE on test set: 1.2019006957203215


Epoch 15/15: 100%|██████████| 4045/4045 [00:56<00:00, 72.03it/s]


Epoch 15/15, Avg. Loss: 0.3495


Testing Epoch 15: 100%|██████████| 1012/1012 [00:04<00:00, 215.31it/s]

Epoch 15/15, RMSE on test set: 1.2066358693356296
